In [1]:
data_var = '2024-01-06'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11368,2024-01-06,Brasil Nbb,16:00,Brasília,Pato,2.48,1.50,155.5,1.81,1.89,1.5,2.10,1.67,8UX8LJBr,0.403226,0.666667,0.552486,0.529101,0.069892,0.6,0.8,NaN,NaN,422.154,168.640720,0.399477,242.224,158.714799,0.655240,175.266,299.948,249.09,501.00,0.0,0.0,0.0,0.0,0.348223,0.030578,0.161303,-2.81,-0.562,-2.633452,0.0,0.0,0.0,-5.00,-1.000,-0.500000,0.0,0.0,0.0
11369,2024-01-06,Brasil Nbb,17:00,São Paulo,Bauru,1.57,2.31,152.5,1.81,1.89,-4.5,1.97,1.73,jgNDKwRl,0.636943,0.432900,0.552486,0.529101,0.069843,0.0,0.8,NaN,NaN,117.700,43.393836,0.368682,132.832,47.686377,0.358998,114.568,168.840,73.20,214.84,0.0,0.0,0.0,0.0,0.269721,0.030578,0.091733,-2.22,-0.444,-1.283784,0.0,0.0,0.0,-0.94,-0.188,-6.968085,0.0,0.0,0.0
11370,2024-01-06,Espanha Acb,14:00,MoraBanc Andorra,Murcia,2.06,1.77,164.5,1.84,1.92,1.5,1.90,1.89,lAdUXinO,0.485437,0.564972,0.543478,0.520833,0.050409,0.0,0.4,NaN,NaN,351.500,202.087866,0.574930,126.866,13.222376,0.104223,0.000,128.870,184.80,147.03,0.0,0.0,0.0,0.0,0.107081,0.030090,0.003731,0.00,0.000,inf,0.0,0.0,0.0,-0.18,-0.036,-21.388889,0.0,0.0,0.0
11371,2024-01-06,Espanha Acb,14:00,Zaragoza,Basquet Girona,1.38,3.08,167.5,1.88,1.92,-7.5,2.01,1.76,fmcQYXWH,0.724638,0.324675,0.531915,0.520833,0.049313,0.6,0.0,NaN,NaN,264.204,107.810229,0.408057,208.784,61.843112,0.296206,226.258,0.000,430.26,220.52,1.0,0.0,0.0,0.0,0.539050,0.014886,0.093781,5.10,1.020,0.372549,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11372,2024-01-06,Espanha Acb,16:45,Palencia,Unicaja,5.15,1.17,162.5,1.85,1.93,9.5,2.07,1.70,S0eYWB1U,0.194175,0.854701,0.540541,0.518135,0.048876,0.0,0.4,NaN,NaN,360.574,254.136381,0.704811,136.140,40.016845,0.293939,0.000,189.844,166.92,163.68,0.0,0.0,0.0,0.0,0.890597,0.029930,0.138795,0.00,0.000,inf,0.0,0.0,0.0,5.67,1.134,0.149912,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11699,2024-01-06,Hungria Nacional Feminino,14:00,Sopron F,TFSE-MTK F,1.22,3.98,140.5,1.85,1.85,-9.5,1.83,1.87,6D3kgSpJ,0.819672,0.251256,0.540541,0.540541,0.070928,0.0,0.0,NaN,NaN,114.322,43.287016,0.378641,NaN,NaN,NaN,0.000,0.000,91.45,133.48,0.0,0.0,0.0,0.0,0.750621,0.000000,0.015289,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11700,2024-01-06,Portugal Proliga,14:00,Santo André,Imortal B,1.19,4.37,160.5,1.80,1.90,-10.5,1.87,1.83,WfXqnZMB,0.840336,0.228833,0.555556,0.526316,0.069169,0.0,0.0,NaN,NaN,183.836,81.440094,0.443004,NaN,NaN,NaN,0.000,0.000,132.00,115.20,0.0,0.0,0.0,0.0,0.808849,0.038222,0.015289,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11701,2024-01-06,Portugal Proliga,15:00,Illiabum,Sampaense,1.11,5.77,152.5,1.83,1.87,-12.5,1.85,1.85,dtaNFkPI,0.900901,0.173310,0.546448,0.534759,0.074211,0.0,0.0,NaN,NaN,126.586,31.285811,0.247151,304.902,104.091399,0.341393,0.000,0.000,102.70,346.26,0.0,0.0,0.0,0.0,0.957883,0.015289,0.000000,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11702,2024-01-06,Portugal Proliga,15:30,Sangalhos,Vasco Da Gama,1.54,2.37,156.5,1.85,1.85,-4.5,1.93,1.78,GQFZILjt,0.649351,0.421941,0.540541,0.540541,0.071292,0.0,0.0,NaN,NaN,144.372,46.391328,0.321332,137.864,40.376051,0.292869,0.000,0.000,124.74,115.90,0.0,0.0,0.0,0.0,0.300204,0.000000,0.057178,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Detroit Pistons,237.5,1.83,1.0,Over
1,00:00,Eua Nba,Los Angeles Lakers,Memphis Grizzlies,224.5,1.86,1.0,Over
2,00:00,Eua Nba,Sacramento Kings,Toronto Raptors,236.5,1.89,1.0,Over
